Econ 202A Macroeconomics \
Fall 2021 \
Jon Steinsson

Problem Set 1 \
October 26, 2021

Steven Zheng \
<steven_zheng@berkeley.edu>

This notebook contains the code and output for parts c through l. Parts a and b are in the file `poly.py`, and part m is in the notebook `hw1_partm.ipynb`.

In [1]:
from Settings import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Part a
Write the household’s problem recursively. Be sure to state what variables are chosen and all the constraints.

The household's utility is
\begin{align}
\mathbb{E}_0 \sum_{t=0}^\infty \beta^t U(C_t)
\end{align}

where 
\begin{align}
U(C_t) = \frac{C_t^{1-\gamma}}{1-\gamma}
\end{align}

subject to the budget constraint
\begin{align}
A_{t+1} = Z(Y_t - C_t + A_t)^\alpha
\end{align}

with labor income
\begin{align}
\log Y_t = (1-\rho) \log \mu + \rho \log Y_{t-1} + \epsilon_t, \quad \epsilon_t \sim N(0,\sigma^2)
\end{align}



We can rewrite the budget constraint as
\begin{align}
A_{t+1}^{1/\alpha} =& Z^{1/\alpha} (Y_t - C_t + A_t)
\\ Z^{1/\alpha} C_t =& Z^{1/\alpha} (Y_t + A_t) - A_{t+1}^{1/\alpha}
\\ C_t =& Y_t + A_t - Z^{-\frac{1}{\alpha}} A_{t+1}^{\frac{1}{\alpha}}
\\ C_t =& Y_t + A_t - \left( \frac{A_{t+1}}{Z} \right)^{1/\alpha}
\end{align}

So we can rewrite the maximization as
\begin{align}
\max_{\{A_{t+1}\}_{t=0}^\infty} \mathbb{E} \sum_{t=0}^\infty \beta^t U \left( Y_t + A_t - \left( \frac{A_{t+1}}{Z} \right)^{1/\alpha} \right)
\end{align}
such that $A_{t+1} \in \Gamma(A_t,Y_t)$, $\forall t$.

And then rewrite from the sequence problem to functional equation
\begin{align}
V^\star(A_0,Y_0) &= \max_{\{A_{t+1}\}_{t=0}^\infty} \mathbb{E} \sum_{t=0}^\infty \beta^t U \left( Y_t + A_t - \left( \frac{A_{t+1}}{Z} \right)^{1/\alpha} \right)
\\ &= \max_{A_{t+1} \in \Gamma(A_t,Y_t), \ \forall t} \mathbb{E} \sum_{t=0}^\infty \beta^t U \left( Y_t + A_t - \left( \frac{A_{t+1}}{Z} \right)^{1/\alpha} \right)
\\ &= \max_{A_{1} \in \Gamma(A_0,Y_0)} \Bigg[ U \left( Y_0 + A_0 - \left( \frac{A_{1}}{Z} \right)^{1/\alpha} \right) 
\\ &+ \beta \max_{A_{(t+1)+1} \in \Gamma(A_{t+1},Y_{t+1}), \ \forall t} \mathbb{E} \sum_{t=0}^\infty \beta^t U \left( Y_{t+1} + A_{t+1} - \left( \frac{A_{(t+1)+1}}{Z} \right)^{1/\alpha} \right) \Bigg]
\\ &= \max_{A_{1} \in \Gamma(A_0,Y_0)} \left[ U \left( Y_0 + A_0 - \left( \frac{A_{1}}{Z} \right)^{1/\alpha} \right) +\beta \mathbb{E} V^{\star}(A_1,Y_1) \right]
\end{align}

so then
\begin{align*}
V(A,\log Y) =& \max_{A^\prime} U \left( Y + A - \left(\frac{A^\prime}{Z}\right)^{(1/\alpha)} \right) + \beta \mathbb{E} \left[ V(A^\prime, \log Y^\prime) \right]
\end{align*}

with labor income
\begin{align}
\log Y_t = (1-\rho) \log \mu + \rho \log Y_{t-1} + \epsilon_t, \quad \epsilon_t \sim N(0,\sigma^2)
\end{align}

The choice variable are $A$ and $Y$. The parameter values are $\gamma=2$, $\beta=0.94$, $\mu=1$, $b=0.4$, $\rho=0.9$, $\sigma^2=0.01$, $r=0.05$, $\alpha=1/3$ and $Z=20$.

## Part b
Write a version of McKay’s PolyBasis function for this problem. Use a 2nd order polynomial basis (which will have 6 terms).

This is the function `PolyBasis()` in the file `poly.py`.

## Part c
Write a version of McKay's PolyGetCoeff function for this problem

This is the function `PolyGetCoef()` in the file `poly.py`.

## Part d
Start your main Matlab program by reading in the parameter values into a structure.

In [2]:
gamma=2
beta=0.94
mu=1
b=0.4
rho=0.9
sigma2=0.01
sigma=np.sqrt(sigma2)
r=0.05
alpha=1/3
Z=20

## Part e
Create a grid on $A$ and $\log Y$. Feel free to use McKay’s `tauchen` function as needed. Use 7 grid points for $\log Y$ and 100 grid points for $A$. (Please create the grid for $\log Y$, not $Y$ . Uniformity will make grading easier.) Calculate the steady-state of $A$ using the first-order conditions of the sequence problem and the budget constraint. Then create an equally spaced grid from $0.05\overline{A}$ to $1.95\overline{A}$, where $\overline{A}$ is the steady state.

Note: I copy Alisdair's `tauchen()` function into Python. It's in `Settings/tauchen.py`.

The sequence problem is
\begin{align}
\max_{\{ C_t, A_t \}} \mathbb{E} \sum_{t=0}^\infty \beta^t \frac{C_t^{1-\gamma}}{1-\gamma}
\end{align}
such that
\begin{align}
A_{t+1} = Z(Y_t - C_t + A_t)^\alpha.
\end{align}

The Lagrangian is
\begin{align}
\mathcal{L} = \mathbb{E} \sum_{t=0}^\infty \beta^t \frac{C_t^{1-\gamma}}{1-\gamma} - \lambda_t \left[A_{t+1} - Z(Y_t - C_t + A_t)^\alpha \right].
\end{align} 

The FOCs are WRT $C_t$
\begin{align}
0&=\frac{\partial \mathcal{L}}{\partial C_t}
\\ &= \beta^t C_t^{-\gamma} - \lambda_t Z \alpha (Y_t - C_t + A_t )^{\alpha -1} 
\\ \iff \beta^t C_t^{-\gamma} &= \lambda_t Z \alpha (Y_t - C_t + A_t )^{\alpha -1}  
\end{align}

The FOCs are WRT $C_t$
\begin{align}
0&=\frac{\partial \mathcal{L}}{\partial C_t}
\\ &= \beta^t C_t^{-\gamma} - \lambda_t Z \alpha (Y_t - C_t + A_t )^{\alpha -1} 
\\ \iff \beta^t C_t^{-\gamma} &= \lambda_t Z \alpha (Y_t - C_t + A_t )^{\alpha -1}
\\ \iff \lambda_t &= \beta
\end{align}

and $A_{t+1}$
\begin{align}
0 &= \frac{\partial \mathcal{L}}{\partial A_{t+1}}
\\ &= \beta^t (-\lambda_{t}) + \beta^{t+1} \mathbb{E} \left[\lambda_{t+1} Z \alpha (Y_{t+1} - C_{t+1} + A_{t+1})^{\alpha-1} \right]
\\ \lambda_t &= \beta \mathbb{E} \left[\lambda_{t+1} Z \alpha (Y_{t+1} - C_{t+1} + A_{t+1})^{\alpha-1} \right]
\end{align}

In [3]:
grid_logY, grid_logYprob = tauchen(N=7,
                                   mu=1.0-rho,
                                   rho=rho,
                                   sigma=sigma,
                                   m=4)
grid_logYprob = grid_logYprob.T

In [ ]:
grid_A = np.linspace(start=0.05*A_bar,
                     stop=1.95*A_bar,
                     num=100)
